In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/access_pattern/all_access_pattern.json"

In [ ]:
import json

results_raw = json.loads(open(RESULT_JSON).read())
assert results_raw["context"]["library_build_type"] != "debug"

BENCHMARKS =  results_raw["benchmarks"]
# print(BENCHMARKS)

In [ ]:
from collections import defaultdict
runs = defaultdict(list)

DIMM = 'dimm'
BLOCK = 'block'

for bm in BENCHMARKS:
    if DIMM in bm['name']:
        runs[DIMM].append(bm)
    elif BLOCK in bm['name']:
        runs[BLOCK].append(bm)
    else:
        raise RuntimeError(f"Unknown benchmark type {bm['name']}")

runs[DIMM].sort(key=lambda x: x['threads'])
runs[BLOCK].sort(key=lambda x: x['threads'])

pprint(runs)

In [ ]:
dimm_runs = [run['items_per_second'] / MILLION for run in runs[DIMM]]
block_runs = [run['items_per_second'] / MILLION for run in runs[BLOCK]]

fig, ax = plt.subplots(1, 1, figsize=(4, 3))

num_threads = list(range(1, 37, ))
ax.plot(num_threads, dimm_runs, label="DIMM", ls='-.', color=COLORS[3])
ax.plot(num_threads, block_runs, label="VBlock", ls='-', color=COLORS[0])
ax.set_yticks(range(1, 12))
ax.yaxis.set_major_locator(MultipleLocator(2))

ax.set_ylabel("Throughput (Mops/s)", fontsize=18)
ax.set_xlabel("# Threads", fontsize=18)
ax.legend(frameon=True, loc='upper left', fontsize=18,
          columnspacing=1.3, handletextpad=0.3, labelspacing=0.1,
         bbox_to_anchor=(-0.03, 1.13))

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(18)

ax.yaxis.set_label_coords(-0.12, 0.45)

ax.set_axisbelow(True)
ax.grid(axis='y', which='major')
fig.savefig('charts/access_pattern.pdf', bbox_inches='tight')
fig.savefig('charts/access_pattern.svg', bbox_inches='tight')